<a 
    target="_blank" 
    href="https://colab.research.google.com/github/davidgasquez/gitcoin-grants-data-portal/blob/main/notebooks/sandbox.ipynb"> 
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Following parquet files can be accessed from `ipfs`, see the **home** page for instructions.

Raw files are direct result of API fetch, while processed files have been cleaned and normalized. 

Current convention for processed files is to store `address` address in lowercase format. Columns are renamed to `snake_case`. 

In [18]:
#| echo: false
import os

import pandas as pd
import pyarrow.parquet as pq
from IPython.display import display, Markdown

In [19]:
#| echo: false

# Make sure to call `make run` first so `tables` folder is actually populated.

EXPORT_PATH = os.path.join('..','tables')
EXTENSIONS  = ('.parquet')

raw_files = []
processed_files = []

for file in  os.listdir(EXPORT_PATH):
    if file.endswith(EXTENSIONS):
        if file.startswith('public_raw'):
            raw_files.append(file)
        else:
            processed_files.append(file)

def print_section(section_header: str, file_list):
    print(section_header)
    for file in file_list:
        print(file)

raw_files = sorted(raw_files)
processed_files = sorted(processed_files)

print_section('Processed Files:', processed_files)
print()
print_section('Raw Files:', raw_files)

Processed Files:
application_answers.parquet
chain_metadata.parquet
passport_scores.parquet
projects.parquet
round_applications.parquet
round_votes.parquet
rounds.parquet

Raw Files:
public_raw_chain_metadata.parquet
public_raw_passport_scores.parquet
public_raw_prices.parquet
public_raw_projects.parquet
public_raw_round_applications.parquet
public_raw_round_contributors.parquet
public_raw_round_votes.parquet
public_raw_rounds.parquet


For each of the files you can find sample rows, and schema below.

## Processed Files

In [20]:
#| echo: false

def md(content):
    display(Markdown(content))

def print_file_sections(files, with_schema=True):
    for file_name in files:
        file_path = os.path.join(EXPORT_PATH, file_name)
        if not os.path.exists(file_path):
            print(f"File not found: {file_name}")
            continue

        md(f'### {file_name}')
        
        if with_schema:
            try:
                schema = pq.read_schema(file_path)
                schema_str = str(schema).replace('\n', '<br>') 
                md(schema_str)

            except Exception as e:
                print('Could not parse:', e) 

        display(pd.read_parquet(file_path).head(4))




print_file_sections(processed_files,with_schema=True)

None

### application_answers.parquet

question_id: string<br>project_id: string<br>application_id: int64<br>chain_id: string<br>round_id: string<br>question: string<br>question_type: string<br>answer: string

,question_id,project_id,application_id,chain_id,round_id,question,question_type,answer
0,0,0xab83cae7352409a46369fdbbcb4f0ebe0cda64d8278a...,0,324,0xc85686f394116ddf94dbfa86059e17047232c54f,Funding Sources,short-answer,None
1,1,0xab83cae7352409a46369fdbbcb4f0ebe0cda64d8278a...,0,324,0xc85686f394116ddf94dbfa86059e17047232c54f,Team Size,number,1
2,0,0x1b014acdbd404fda33ec62e3c82d8575f5eef50a3d6e...,1,324,0xc85686f394116ddf94dbfa86059e17047232c54f,Funding Sources,short-answer,2
3,1,0x1b014acdbd404fda33ec62e3c82d8575f5eef50a3d6e...,1,324,0xc85686f394116ddf94dbfa86059e17047232c54f,Team Size,number,2


### chain_metadata.parquet

name: string<br>short_name: string<br>info_url: string<br>chain_id: string<br>network_id: int64<br>native_currency: struct<name: string, symbol: string, decimals: int32><br>  child 0, name: string<br>  child 1, symbol: string<br>  child 2, decimals: int32<br>ens: struct<registry: string><br>  child 0, registry: string<br>explorers: list<element: map<string, string ('element')>><br>  child 0, element: map<string, string ('element')><br>      child 0, element: struct<key: string not null, value: string> not null<br>          child 0, key: string not null<br>          child 1, value: string<br>rpc: list<element: string><br>  child 0, element: string<br>parent: struct<type: string, chain: string, bridges: list<element: struct<url: string>>><br>  child 0, type: string<br>  child 1, chain: string<br>  child 2, bridges: list<element: struct<url: string>><br>      child 0, element: struct<url: string><br>          child 0, url: string

...

chain_id: string<br>round_id: string<br>id: string<br>transaction: string<br>block_number: int64<br>project_id: string<br>application_id: int64<br>voter: string<br>grant_address: string<br>token: string<br>amount: double<br>amount_usd: double<br>amount_round_token: double

,chain_id,round_id,id,transaction,block_number,project_id,application_id,voter,grant_address,token,amount,amount_usd,amount_round_token
0,137,0xa1d52f9b5339792651861329a046dd912761e9a9,0x5f06f13f85a40e4882cfb33dcd349d2b7f7a87784706...,0xf99910d3cd1e0bde4427bfd62ec20063c10a83a8fcc1...,49973112,0xaa09ef03fe5e4d9a0aa3d49a203ea8d1d89969e19cfe...,14,0x1dceccd9597290c3f86dd17b09c13bfbae925e17,0x6e41fe2f8303b89c9dbccabe59a7f7f8f4312ca9,0x0000000000000000000000000000000000000000,2.000000e+18,1.895590,1894791.0
1,137,0xa1d52f9b5339792651861329a046dd912761e9a9,0xc6007eee11e683285f67e5a804e1ef2e15e41ae97b19...,0xc15552dffdb15530d3f20cab14489b94eb2ae31bd307...,49973170,0xad02fe3124cef38f8c4fd5a44302713e49458d60a84d...,17,0x75c8cd900faa01842f19093437d368a7b41a8471,0x35292bbd442373046a6fb4bf7e2529fa9a8d5bb0,0x0000000000000000000000000000000000000000,1.050000e+18,0.995185,994765.0
2,137,0xa1d52f9b5339792651861329a046dd912761e9a9,0xbcb01eb7a45eb51f16cbcbc9470e6d3589efa69a9172...,0x8572e49040d21067e02fd6b4cf49076e43a094b2ca71...,49973173,0xb510aeabc101c97d8612650059a43a1b4ed1c8c86e96...,9,0x4a3755eb99ae8b22aafb8f16f0c51cf68eb60b85,0x242ba6d68ffeb4a098b591b32d370f973ff882b7,0x0000000000000000000000000000000000000000,1.052630e+18,0.997678,997257.0
3,137,0xa1d52f9b5339792651861329a046dd912761e9a9,0x707cc5d56bbefaa0bffcbbc329e79f64017bb7b3e5f7...,0xf1c397143eb68d525fb280609f3b77fbc24c8f4f7ff0...,49973181,0xb510aeabc101c97d8612650059a43a1b4ed1c8c86e96...,9,0x88d7709ce401e4e7b5068156423ecb4f60a99f75,0x242ba6d68ffeb4a098b591b32d370f973ff882b7,0x0000000000000000000000000000000000000000,1.053265e+18,0.998279,997858.0


### rounds.parquet

id: string<br>amount_usd: double<br>votes: int64<br>token: string<br>match_amount: double<br>match_amount_usd: double<br>unique_contributors: int64<br>application_meta_ptr: string<br>application_metadata: string<br>meta_ptr: string<br>metadata: string<br>applications_start_time: int64<br>applications_end_time: double<br>round_start_time: int64<br>round_end_time: double<br>created_at_block: int64<br>updated_at_block: int64<br>chain_id: string<br>name: string<br>round_type: string<br>program_address: string<br>sybil_defense: bool

,id,amount_usd,votes,token,match_amount,match_amount_usd,unique_contributors,application_meta_ptr,application_metadata,meta_ptr,...,applications_end_time,round_start_time,round_end_time,created_at_block,updated_at_block,chain_id,name,round_type,program_address,sybil_defense
0,0x40b60e358026e9a72e9af11a4a7d9be84ea0d60d,0.000000,0,0x0000000000000000000000000000000000000000,1.000000e+17,185.523225,0,bafkreidiwkz5ehuohzl2rpexl5boxjqkwp5yvvdgaxpg6...,"{'lastUpdatedOn': 1680876301552, 'applicationS...",bafkreigenkg7oza44daclqlpccmopzk3ubajgdhdq5fhy...,...,1.681085e+09,1681171200,1.681344e+09,16997091,16997091,1,Test Round 1,None,0x16a300facb87ea5e486cd1ec1201804610d281b1,None
1,0x87306d9cc4fd16b702470e0ebee3c05162ff238b,0.000000,0,0x6b175474e89094c44da98b954eedeac495271d0f,1.000000e+21,999.975900,0,bafkreiergecz32x3kcnevra66kosnbvzyo74ygdbusvit...,"{'lastUpdatedOn': 1681220170629, 'applicationS...",bafkreieuq6n7dlu3im762rfajsx6ekqc4yclwzldzanhb...,...,1.681221e+09,1681221000,1.681344e+09,17025192,17025192,1,Testing round,None,0x009cd137f93b1d125f5109ebaee10c20075762ff,True
2,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,330743.553898,81867,0x6b175474e89094c44da98b954eedeac495271d0f,3.500000e+23,350067.210500,14554,bafkreigliumevo4mbmoezolhl7opurang42gspvir7lkm...,"{'lastUpdatedOn': 1681254207125, 'applicationS...",bafkreiaejlsquht5smr45klyr6yhc7s66rljoqtwrkq55...,...,1.682122e+09,1682424000,1.683677e+09,17027966,17027966,1,Web3 Open Source Software Round,None,0xa1b6245d7ba4b126adf7ee1e05e96bfda974990c,True
3,0xdf22a2c8f6ba9376ff17ee13e6154b784ee92094,42224.337015,4307,0x6b175474e89094c44da98b954eedeac495271d0f,2.000000e+23,200038.406000,1062,bafkreibertcivnryfjfxd2navihysw5l76mqmqsmf2fkg...,"{'lastUpdatedOn': 1681256584092, 'applicationS...",bafkreifcxurlia7wbjekx2bd3q7disqdp5teiyd46qi74...,...,1.682122e+09,1682424000,1.683677e+09,17028163,17028163,1,Ethereum Infrastructure,None,0xa1b6245d7ba4b126adf7ee1e05e96bfda974990c,True


## Raw Files

In [21]:
#| echo: false
print_file_sections(raw_files,with_schema=True)

### public_raw_chain_metadata.parquet

name: string<br>chain: string<br>icon: string<br>rpc: list<element: string><br>  child 0, element: string<br>features: list<element: struct<name: string>><br>  child 0, element: struct<name: string><br>      child 0, name: string<br>faucets: list<element: int32><br>  child 0, element: int32<br>nativeCurrency: struct<name: string, symbol: string, decimals: int32><br>  child 0, name: string<br>  child 1, symbol: string<br>  child 2, decimals: int32<br>infoURL: string<br>shortName: string<br>chainId: string<br>networkId: int64<br>slip44: int64<br>ens: struct<registry: string><br>  child 0, registry: string<br>explorers: list<element: map<string, string ('element')>><br>  child 0, element: map<string, string ('element')><br>      child 0, element: struct<key: string not null, value: string> not null<br>          child 0, key: string not null<br>          child 1, value: string<br>title: string<br>status: string<br>redFlags: int32<br>parent: struct<type: string, chain: string, bridges: list<element: struct<url: string>>><br>  child 0, type: string<br>  child 1, chain: string<br>  child 2, bridges: list<element: struct<url: string>><br>      child 0, element: struct<url: string><br>          child 0, url: string

,name,chain,icon,rpc,features,faucets,nativeCurrency,infoURL,shortName,chainId,networkId,slip44,ens,explorers,title,status,redFlags,parent
0,Ethereum Mainnet,ETH,ethereum,[https://mainnet.infura.io/v3/${INFURA_API_KEY...,"[{'name': 'EIP155'}, {'name': 'EIP1559'}]",[],"{'name': 'Ether', 'symbol': 'ETH', 'decimals':...",https://ethereum.org,eth,1,1,60.0,{'registry': '0x00000000000C2E074eC69A0dFb2997...,"[[(name, etherscan), (url, https://etherscan.i...",None,None,NaN,None
1,OP Mainnet,ETH,None,"[https://mainnet.optimism.io, https://optimism...",None,[],"{'name': 'Ether', 'symbol': 'ETH', 'decimals':...",https://optimism.io,oeth,10,10,NaN,None,"[[(name, etherscan), (url, https://optimistic....",None,None,NaN,None
2,Polygon Mainnet,Polygon,polygon,"[https://polygon-rpc.com/, https://rpc-mainnet...",None,[],"{'name': 'MATIC', 'symbol': 'MATIC', 'decimals...",https://polygon.technology/,matic,137,137,966.0,None,"[[(name, polygonscan), (url, https://polygonsc...",None,None,NaN,None
3,Fantom Opera,FTM,fantom,"[https://rpc.ftm.tools, https://fantom.publicn...",None,[],"{'name': 'Fantom', 'symbol': 'FTM', 'decimals'...",https://fantom.foundation,ftm,250,250,NaN,None,"[[(name, ftmscan), (url, https://ftmscan.com),...",None,None,NaN,None


### public_raw_passport_scores.parquet

address: string<br>score: int64<br>last_score_timestamp: timestamp[us, tz=UTC]<br>status: string<br>evidence: struct<type: string, success: bool, rawScore: string, threshold: string><br>  child 0, type: string<br>  child 1, success: bool<br>  child 2, rawScore: string<br>  child 3, threshold: string<br>id: int64

...

chainId: string<br>roundId: string<br>id: string<br>transaction: string<br>blockNumber: int64<br>projectId: string<br>applicationId: int64<br>voter: string<br>grantAddress: string<br>token: string<br>amount: double<br>amountUSD: double<br>amountRoundToken: double

,chainId,roundId,id,transaction,blockNumber,projectId,applicationId,voter,grantAddress,token,amount,amountUSD,amountRoundToken
0,137,0xa1D52F9b5339792651861329A046dD912761E9A9,0x5f06f13f85a40e4882cfb33dcd349d2b7f7a87784706...,0xf99910d3cd1e0bde4427bfd62ec20063c10a83a8fcc1...,49973112,0xaa09ef03fe5e4d9a0aa3d49a203ea8d1d89969e19cfe...,14,0x1dCecCD9597290c3f86Dd17b09c13BfbAE925e17,0x6E41fe2F8303b89c9dbcCABE59A7F7f8F4312cA9,0x0000000000000000000000000000000000000000,2.000000e+18,1.895590,1894791.0
1,137,0xa1D52F9b5339792651861329A046dD912761E9A9,0xc6007eee11e683285f67e5a804e1ef2e15e41ae97b19...,0xc15552dffdb15530d3f20cab14489b94eb2ae31bd307...,49973170,0xad02fe3124cef38f8c4fd5a44302713e49458d60a84d...,17,0x75c8CD900fAA01842f19093437D368a7B41a8471,0x35292bbd442373046a6fb4bF7e2529FA9A8D5Bb0,0x0000000000000000000000000000000000000000,1.050000e+18,0.995185,994765.0
2,137,0xa1D52F9b5339792651861329A046dD912761E9A9,0xbcb01eb7a45eb51f16cbcbc9470e6d3589efa69a9172...,0x8572e49040d21067e02fd6b4cf49076e43a094b2ca71...,49973173,0xb510aeabc101c97d8612650059a43a1b4ed1c8c86e96...,9,0x4a3755eB99ae8b22AaFB8f16F0C51CF68Eb60b85,0x242ba6d68FfEb4a098B591B32d370F973FF882B7,0x0000000000000000000000000000000000000000,1.052630e+18,0.997678,997257.0
3,137,0xa1D52F9b5339792651861329A046dD912761E9A9,0x707cc5d56bbefaa0bffcbbc329e79f64017bb7b3e5f7...,0xf1c397143eb68d525fb280609f3b77fbc24c8f4f7ff0...,49973181,0xb510aeabc101c97d8612650059a43a1b4ed1c8c86e96...,9,0x88d7709ce401e4E7b5068156423ECB4f60A99F75,0x242ba6d68FfEb4a098B591B32d370F973FF882B7,0x0000000000000000000000000000000000000000,1.053265e+18,0.998279,997858.0


### public_raw_rounds.parquet

id: string<br>amountUSD: double<br>votes: int64<br>token: string<br>matchAmount: double<br>matchAmountUSD: double<br>uniqueContributors: int64<br>applicationMetaPtr: string<br>applicationMetadata: string<br>metaPtr: string<br>metadata: string<br>applicationsStartTime: int64<br>applicationsEndTime: double<br>roundStartTime: int64<br>roundEndTime: double<br>createdAtBlock: int64<br>updatedAtBlock: int64<br>chainId: string

,id,amountUSD,votes,token,matchAmount,matchAmountUSD,uniqueContributors,applicationMetaPtr,applicationMetadata,metaPtr,metadata,applicationsStartTime,applicationsEndTime,roundStartTime,roundEndTime,createdAtBlock,updatedAtBlock,chainId
0,0x40b60e358026e9a72e9Af11a4A7D9bE84Ea0d60d,0.000000,0,0x0000000000000000000000000000000000000000,1.000000e+17,185.523225,0,bafkreidiwkz5ehuohzl2rpexl5boxjqkwp5yvvdgaxpg6...,"{'lastUpdatedOn': 1680876301552, 'applicationS...",bafkreigenkg7oza44daclqlpccmopzk3ubajgdhdq5fhy...,"{""matchingFunds"": {""matchingCap"": false, ""matc...",1680877800,1.681085e+09,1681171200,1.681344e+09,16997091,16997091,1
1,0x87306D9Cc4fD16b702470e0eBEE3C05162fF238B,0.000000,0,0x6b175474e89094c44da98b954eedeac495271d0f,1.000000e+21,999.975900,0,bafkreiergecz32x3kcnevra66kosnbvzyo74ygdbusvit...,"{'lastUpdatedOn': 1681220170629, 'applicationS...",bafkreieuq6n7dlu3im762rfajsx6ekqc4yclwzldzanhb...,"{""feesPercentage"": 0, ""feesAddress"": """", ""name...",1681220400,1.681221e+09,1681221000,1.681344e+09,17025192,17025192,1
2,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,330743.553898,81867,0x6b175474e89094c44da98b954eedeac495271d0f,3.500000e+23,350067.210500,14554,bafkreigliumevo4mbmoezolhl7opurang42gspvir7lkm...,"{'lastUpdatedOn': 1681254207125, 'applicationS...",bafkreiaejlsquht5smr45klyr6yhc7s66rljoqtwrkq55...,"{""feesPercentage"": 0, ""feesAddress"": """", ""name...",1681257600,1.682122e+09,1682424000,1.683677e+09,17027966,17027966,1
3,0xdf22a2C8F6BA9376fF17EE13E6154B784ee92094,42224.337015,4307,0x6b175474e89094c44da98b954eedeac495271d0f,2.000000e+23,200038.406000,1062,bafkreibertcivnryfjfxd2navihysw5l76mqmqsmf2fkg...,"{'lastUpdatedOn': 1681256584092, 'applicationS...",bafkreifcxurlia7wbjekx2bd3q7disqdp5teiyd46qi74...,"{""feesPercentage"": 0, ""feesAddress"": """", ""name...",1681257600,1.682122e+09,1682424000,1.683677e+09,17028163,17028163,1
